In [4]:
import gym
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch as T

In [8]:
env = gym.make('CartPole-v1')

In [6]:
class PolicyNet(nn.Module):
    def __init__(self,lr,input_dims,num_actions):
        super(PolicyNet,self).__init__()
        self.fc1 = nn.Linear(*input_dims,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,num_actions)
        self.optimizer = optim.Adam(self.parameters(),lr=lr)
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self,state):
        S = F.relu(self.fc1(state))
        S = F.relu(self.fc2(S))
        S = F.relu(self.fc3(S))
        S = self.fc4(S)
        
        return S

In [17]:
class Reinforce_Agent():
    def __init__(self,lr,input_dims,gamma=0.99,num_actions=2):
        self.gamma = gamma
        self.lr = lr
        self.reward_memory = []
        self.action_memory = []
        
        self.policy = PolicyNet(self.lr,input_dims,num_actions)
        
    def select_action(self,obs):
        state = T.Tensor([obs]).to(self.policy.device)
        probs = F.softmax(self.policy.forward(state))
        action_probs = T.distributions.Categorical(probs)
        action = action_probs.sample()
        log_probs = action_probs.log_prob(action)
        self.action_memory.append(log_probs)
        return action.item()
    def store_rewards(self,reward):
        self.reward_memory.append(reward)
    
    def train(self):
        self.policy.optimizer.zero_grad()
        G = np.zeros_like(self.reward_memory,dtype = np.float64)
        for t in range(len(self.reward_memory)):
            G_sum = 0
            discount = 1
            for k in range(t,len(self.reward_memory)):
                G_sum += self.reward_memory[k] * discount
                discount *= self.gamma
            G[t] = G_sum
        G = T.tensor(G,dtype = T.float).to(self.policy.device)
        loss = 0
        for g,logprob in zip(G,self.action_memory):
            loss += -g*logprob
        loss.backward()
        self.policy.optimizer.step()
        
        self.action_memory = []
        self.reward_memory = []
        

In [18]:
epochs = 3000
agent = Reinforce_Agent(gamma= 0.99,lr = 0.0005,input_dims=[4],num_actions =2)
scores = []
for epoch in range(epochs):
    done = False
    obs = env.reset()
    score = 0
    while not done:
        action = agent.select_action(obs)
        next_obs,reward,done,info = env.step(action)
        score+=reward
        agent.store_rewards(reward)
        obs = next_obs
    agent.train()
    scores.append(score)
    avg_score = np.mean(scores[-100:])
    print('Episode: {} Score: {} average acore: {}'.format(epoch,score,avg_score))

<ipython-input-17-f6e3572a8035>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(self.policy.forward(state))


Episode: 0 Score: 20.0 average acore: 20.0
Episode: 1 Score: 11.0 average acore: 15.5
Episode: 2 Score: 25.0 average acore: 18.666666666666668
Episode: 3 Score: 31.0 average acore: 21.75
Episode: 4 Score: 15.0 average acore: 20.4
Episode: 5 Score: 39.0 average acore: 23.5
Episode: 6 Score: 16.0 average acore: 22.428571428571427
Episode: 7 Score: 14.0 average acore: 21.375
Episode: 8 Score: 17.0 average acore: 20.88888888888889
Episode: 9 Score: 17.0 average acore: 20.5
Episode: 10 Score: 21.0 average acore: 20.545454545454547
Episode: 11 Score: 12.0 average acore: 19.833333333333332
Episode: 12 Score: 20.0 average acore: 19.846153846153847
Episode: 13 Score: 16.0 average acore: 19.571428571428573
Episode: 14 Score: 47.0 average acore: 21.4
Episode: 15 Score: 18.0 average acore: 21.1875
Episode: 16 Score: 22.0 average acore: 21.235294117647058
Episode: 17 Score: 16.0 average acore: 20.944444444444443
Episode: 18 Score: 18.0 average acore: 20.789473684210527
Episode: 19 Score: 75.0 avera

Episode: 166 Score: 171.0 average acore: 36.26
Episode: 167 Score: 86.0 average acore: 37.03
Episode: 168 Score: 80.0 average acore: 37.62
Episode: 169 Score: 104.0 average acore: 38.54
Episode: 170 Score: 34.0 average acore: 38.68
Episode: 171 Score: 37.0 average acore: 38.62
Episode: 172 Score: 60.0 average acore: 39.13
Episode: 173 Score: 89.0 average acore: 39.85
Episode: 174 Score: 36.0 average acore: 40.08
Episode: 175 Score: 119.0 average acore: 41.02
Episode: 176 Score: 34.0 average acore: 41.16
Episode: 177 Score: 114.0 average acore: 42.11
Episode: 178 Score: 60.0 average acore: 42.39
Episode: 179 Score: 42.0 average acore: 42.65
Episode: 180 Score: 64.0 average acore: 43.03
Episode: 181 Score: 23.0 average acore: 42.57
Episode: 182 Score: 55.0 average acore: 42.98
Episode: 183 Score: 71.0 average acore: 43.5
Episode: 184 Score: 109.0 average acore: 44.34
Episode: 185 Score: 40.0 average acore: 44.6
Episode: 186 Score: 40.0 average acore: 44.85
Episode: 187 Score: 74.0 averag

Episode: 342 Score: 303.0 average acore: 145.39
Episode: 343 Score: 106.0 average acore: 146.08
Episode: 344 Score: 260.0 average acore: 148.41
Episode: 345 Score: 283.0 average acore: 150.52
Episode: 346 Score: 303.0 average acore: 153.14
Episode: 347 Score: 188.0 average acore: 154.51
Episode: 348 Score: 313.0 average acore: 157.34
Episode: 349 Score: 194.0 average acore: 158.78
Episode: 350 Score: 317.0 average acore: 161.67
Episode: 351 Score: 500.0 average acore: 165.81
Episode: 352 Score: 430.0 average acore: 169.81
Episode: 353 Score: 203.0 average acore: 171.13
Episode: 354 Score: 500.0 average acore: 175.57
Episode: 355 Score: 500.0 average acore: 179.98
Episode: 356 Score: 500.0 average acore: 183.87
Episode: 357 Score: 500.0 average acore: 188.31
Episode: 358 Score: 428.0 average acore: 192.13
Episode: 359 Score: 500.0 average acore: 196.47
Episode: 360 Score: 483.0 average acore: 200.54
Episode: 361 Score: 217.0 average acore: 202.35
Episode: 362 Score: 297.0 average acore:

Episode: 514 Score: 490.0 average acore: 266.91
Episode: 515 Score: 353.0 average acore: 269.06
Episode: 516 Score: 500.0 average acore: 272.84
Episode: 517 Score: 362.0 average acore: 274.83
Episode: 518 Score: 451.0 average acore: 277.59
Episode: 519 Score: 500.0 average acore: 281.19
Episode: 520 Score: 500.0 average acore: 284.7
Episode: 521 Score: 380.0 average acore: 286.81
Episode: 522 Score: 500.0 average acore: 289.5
Episode: 523 Score: 500.0 average acore: 292.65
Episode: 524 Score: 251.0 average acore: 293.15
Episode: 525 Score: 500.0 average acore: 296.01
Episode: 526 Score: 500.0 average acore: 298.84
Episode: 527 Score: 500.0 average acore: 302.63
Episode: 528 Score: 500.0 average acore: 306.57
Episode: 529 Score: 500.0 average acore: 308.56
Episode: 530 Score: 500.0 average acore: 311.14
Episode: 531 Score: 500.0 average acore: 313.92
Episode: 532 Score: 500.0 average acore: 316.7
Episode: 533 Score: 500.0 average acore: 319.29
Episode: 534 Score: 500.0 average acore: 32

...

Episode: 2209 Score: 500.0 average acore: 497.12
Episode: 2210 Score: 500.0 average acore: 497.12
Episode: 2211 Score: 500.0 average acore: 497.12
Episode: 2212 Score: 500.0 average acore: 497.12
Episode: 2213 Score: 500.0 average acore: 497.12
Episode: 2214 Score: 500.0 average acore: 497.12
Episode: 2215 Score: 500.0 average acore: 497.12
Episode: 2216 Score: 500.0 average acore: 497.12
Episode: 2217 Score: 500.0 average acore: 497.12
Episode: 2218 Score: 500.0 average acore: 497.12
Episode: 2219 Score: 500.0 average acore: 497.12
Episode: 2220 Score: 500.0 average acore: 497.12
Episode: 2221 Score: 500.0 average acore: 497.12
Episode: 2222 Score: 500.0 average acore: 497.12
Episode: 2223 Score: 500.0 average acore: 497.12
Episode: 2224 Score: 500.0 average acore: 497.12
Episode: 2225 Score: 500.0 average acore: 497.12
Episode: 2226 Score: 500.0 average acore: 497.12
Episode: 2227 Score: 500.0 average acore: 497.12
Episode: 2228 Score: 500.0 average acore: 497.12
Episode: 2229 Score:

Episode: 2377 Score: 500.0 average acore: 499.3
Episode: 2378 Score: 500.0 average acore: 499.3
Episode: 2379 Score: 500.0 average acore: 499.3
Episode: 2380 Score: 500.0 average acore: 499.3
Episode: 2381 Score: 500.0 average acore: 499.3
Episode: 2382 Score: 500.0 average acore: 499.3
Episode: 2383 Score: 472.0 average acore: 499.02
Episode: 2384 Score: 500.0 average acore: 499.02
Episode: 2385 Score: 500.0 average acore: 499.02
Episode: 2386 Score: 500.0 average acore: 499.02
Episode: 2387 Score: 500.0 average acore: 499.02
Episode: 2388 Score: 500.0 average acore: 499.02
Episode: 2389 Score: 500.0 average acore: 499.02
Episode: 2390 Score: 500.0 average acore: 499.02
Episode: 2391 Score: 500.0 average acore: 499.02
Episode: 2392 Score: 500.0 average acore: 499.02
Episode: 2393 Score: 500.0 average acore: 499.02
Episode: 2394 Score: 500.0 average acore: 499.02
Episode: 2395 Score: 500.0 average acore: 499.02
Episode: 2396 Score: 500.0 average acore: 499.02
Episode: 2397 Score: 500.0

Episode: 2545 Score: 391.0 average acore: 488.62
Episode: 2546 Score: 500.0 average acore: 488.62
Episode: 2547 Score: 500.0 average acore: 488.62
Episode: 2548 Score: 474.0 average acore: 488.36
Episode: 2549 Score: 500.0 average acore: 488.36
Episode: 2550 Score: 500.0 average acore: 488.36
Episode: 2551 Score: 500.0 average acore: 488.36
Episode: 2552 Score: 500.0 average acore: 488.36
Episode: 2553 Score: 500.0 average acore: 488.36
Episode: 2554 Score: 476.0 average acore: 488.12
Episode: 2555 Score: 500.0 average acore: 488.12
Episode: 2556 Score: 500.0 average acore: 488.12
Episode: 2557 Score: 500.0 average acore: 488.12
Episode: 2558 Score: 500.0 average acore: 488.12
Episode: 2559 Score: 401.0 average acore: 487.13
Episode: 2560 Score: 343.0 average acore: 485.56
Episode: 2561 Score: 318.0 average acore: 483.74
Episode: 2562 Score: 377.0 average acore: 482.51
Episode: 2563 Score: 500.0 average acore: 482.51
Episode: 2564 Score: 456.0 average acore: 482.07
Episode: 2565 Score:

Episode: 2713 Score: 500.0 average acore: 498.13
Episode: 2714 Score: 500.0 average acore: 498.13
Episode: 2715 Score: 500.0 average acore: 498.13
Episode: 2716 Score: 500.0 average acore: 498.13
Episode: 2717 Score: 409.0 average acore: 497.22
Episode: 2718 Score: 500.0 average acore: 497.22
Episode: 2719 Score: 500.0 average acore: 497.22
Episode: 2720 Score: 500.0 average acore: 497.22
Episode: 2721 Score: 500.0 average acore: 497.22
Episode: 2722 Score: 500.0 average acore: 497.22
Episode: 2723 Score: 500.0 average acore: 497.22
Episode: 2724 Score: 500.0 average acore: 497.22
Episode: 2725 Score: 500.0 average acore: 497.22
Episode: 2726 Score: 500.0 average acore: 497.22
Episode: 2727 Score: 500.0 average acore: 497.22
Episode: 2728 Score: 500.0 average acore: 497.22
Episode: 2729 Score: 500.0 average acore: 497.22
Episode: 2730 Score: 500.0 average acore: 497.22
Episode: 2731 Score: 495.0 average acore: 497.17
Episode: 2732 Score: 500.0 average acore: 497.17
Episode: 2733 Score:

Episode: 2881 Score: 500.0 average acore: 494.28
Episode: 2882 Score: 500.0 average acore: 494.28
Episode: 2883 Score: 500.0 average acore: 494.28
Episode: 2884 Score: 500.0 average acore: 494.28
Episode: 2885 Score: 500.0 average acore: 494.28
Episode: 2886 Score: 500.0 average acore: 494.28
Episode: 2887 Score: 500.0 average acore: 494.28
Episode: 2888 Score: 500.0 average acore: 494.28
Episode: 2889 Score: 500.0 average acore: 494.28
Episode: 2890 Score: 500.0 average acore: 494.28
Episode: 2891 Score: 500.0 average acore: 494.28
Episode: 2892 Score: 500.0 average acore: 494.28
Episode: 2893 Score: 500.0 average acore: 494.28
Episode: 2894 Score: 500.0 average acore: 494.28
Episode: 2895 Score: 458.0 average acore: 493.86
Episode: 2896 Score: 500.0 average acore: 493.86
Episode: 2897 Score: 500.0 average acore: 493.86
Episode: 2898 Score: 432.0 average acore: 493.18
Episode: 2899 Score: 500.0 average acore: 493.18
Episode: 2900 Score: 500.0 average acore: 493.18
Episode: 2901 Score:

In [19]:
env.reset()
for epoch in range(20):
    done = False
    obs = env.reset()
    score = 0
    while not done:
        env.render()
        action = agent.select_action(obs)
        new_obs,reward,done,info = env.step(action)
        score += reward
        agent.store_rewards(reward)
        obs = new_obs
    agent.train()
    scores.append(score)
    #avg_score = np.mean(scores[-100:])
    print('episode: {} score: {}'.format(epoch+1,score))

<ipython-input-17-f6e3572a8035>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(self.policy.forward(state))


episode: 0 score: 500.0
episode: 1 score: 500.0
episode: 2 score: 500.0
episode: 3 score: 500.0
episode: 4 score: 500.0
episode: 5 score: 500.0
episode: 6 score: 500.0
episode: 7 score: 500.0
episode: 8 score: 500.0
episode: 9 score: 500.0
episode: 10 score: 500.0
episode: 11 score: 500.0
episode: 12 score: 500.0
episode: 13 score: 500.0
episode: 14 score: 500.0
episode: 15 score: 500.0
episode: 16 score: 500.0
episode: 17 score: 500.0
episode: 18 score: 500.0
episode: 19 score: 500.0
